# Extracción con petl

![Actividades ETL](./img/etl_actividades.svg)

## Librería petl

- [petl](https://petl.readthedocs.io/en/stable/index.html) es un paquete de propósito general para la extracción, transformación y carga de tablas de datos.
- Diseñado especialmente para trabajar __datos desconocidos, heterogéneos y/o de calidad mixta__.
- __No tiene dependencias__ que no sean módulos propios del nucleo de Python.
- Las _pipelines_ (transformaciones de flujo de datos en un proceso comprendido por varias fases secuenciales) hacen __uso mínimo de la memoria del sistema__ y pueden __escalar a millones de filas si la velocidad no es una prioridad__.
- En general, __los flujos de trabajo no se ejecutan hasta que se soliciten los datos__.
- Soporta programación funcional y orientada a objetos.
-  [Otras alternativas para diseñar procesos ETL](https://petl.readthedocs.io/en/stable/related_work.html)

[Instalación de la librería petl](https://petl.readthedocs.io/en/stable/intro.html#installation)

## Contenedores de datos (tablas)

Los contenedores de datos son __objetos de datos tabulares__ que sorportan __iteración orientada a filas de datos__.

Un iterador sobre un contenedor,
- Retorna un item como una __secuencia__.
- El primer item, con índice `0`, es el encabezado, tipicamente de tipo `str`.
- Cada item subsiguiente es una secuencia de valores.

Un __contenedor desde una lista__, puede ser definida a partir de:
- `list` una lista de listas, donde cada lista es una columna.
- `header` una lista con los nombres de los campos.
- `missing` valor para celdas vacías.

In [3]:
import petl as etl

cols = [[0, 1, 2, 3], ['a', 'b', 'c', 'a'], ['A', 'B', 'C']]  # lista python
tabla = etl.fromcolumns(cols, header=['num', 'minus.', 'mayus.'], missing=None)
print(tabla)

+-----+--------+--------+
| num | minus. | mayus. |
+=====+========+========+
|   0 | a      | A      |
+-----+--------+--------+
|   1 | b      | B      |
+-----+--------+--------+
|   2 | c      | C      |
+-----+--------+--------+
|   3 | a      | None   |
+-----+--------+--------+



## Operaciones básicas sobre filas y columnas de contenedores

Para __seleccionar un rango de filas__ (*slice*), la función `.rowslice(table, from, to, step)` recibe como argumento,
- `table` la tabla objetivo.
- `start, end` rango de celdas exluyendo la primera e incluyendo el último.
- `step` valor incremental de avance del índice, por defecto es `1`.

In [5]:
tabla1 = etl.rowslice(tabla, 1, 4, 2)
print(tabla1)

+-----+--------+--------+
| num | minus. | mayus. |
+=====+========+========+
|   1 | b      | B      |
+-----+--------+--------+
|   3 | a      | None   |
+-----+--------+--------+



Para la __selección de columnas__, la función `.cut(table, col1, col2,..., coln)` permite seleccionarlas a partir de:
- `int` índice de columna. 
- `str` nombre del campo.
- `range` rango de columnas.

In [5]:
tabla2 = etl.cut(tabla, 0, 'minus.')
tabla2

num,minus.
0,a
1,b
2,c
3,a


Para __remover columnas__ a partir de la identificación de cada una de ellas en el argumento de la función `.cutout(table, col#)`.

In [17]:
tabla3 = etl.cutout(tabla, 'minus.')
tabla3

num,mayus.
0,A
1,B
2,C
3,None


Otras [operaciones básicas](https://petl.readthedocs.io/en/stable/transform.html#basic-transformations):
- `.movefield` cambiar posición de columnas.
- `.cat` concatenar tablas.
- `.stack` Apilar tablas.
- `.addfield` agregar campos con datos derivados.
- `.addcolumn` agregar columna con nuevos datos.
- `.addrownumbers` agregar columna con número de filas.
- ...

## Manipulación de encabezados

La función `.rename(table, origen, destino)` __renombra columnas__ apartir del nombre `origen` por `destino`.

In [6]:
tabla4 = etl.rename(tabla, 'minus.', 'minusculas')
tabla4

num,minusculas,mayus.
0,a,A
1,b,B
2,c,C
3,a,None


De otra forma, permite __renombrar un conjunto de columnas__ pasando como argumento un diccionario donde el conjunto `key:value` corresponde al conjunto `origen:destino`.

In [7]:
tabla5 = etl.rename(tabla, {'minus.': 'minusculas', 'mayus.': 'mayusculas'})
tabla5

num,minusculas,mayusculas
0,a,A
1,b,B
2,c,C
3,a,None


La función `.setheader(table, headers)` __reemplaza los nombres de columnas__ actuales por nuevos nombres pasados como `list`.

In [8]:
tabla6 = etl.setheader(tabla, ['NUM', 'LOWER', 'UPPER'])
tabla6

NUM,LOWER,UPPER
0,a,A
1,b,B
2,c,C
3,a,None


Otras [operaciones con encabezados](https://petl.readthedocs.io/en/stable/transform.html#header-manipulations) de contenedores:
- `.sortheader` ordenar columnas
- `.prefixheader` y `sufixheader` agregar prefijos y sufijos todos los nombres de columnas.
- ...

## Extracción desde fuentes

Funciones de tipo `.from...`

### Archivos de texto en formato plano

- Archivos compuestos de __bytes que representan caracteres__ (letras, números, símbolos, de control).
- Los caracteres pueden estar __codificados en distintos modos__ (e.g. ASCII, UTF-8, Unicode).
- Este tipo de archivo __es legible por el usuario__ y pueden ser editados por medio de editores de texto.
- Extensiones habituales: `txt`, `asc`, `xyz`, `dat`, etc,...

__Para extraer datos desde un archivo de texto__, la función `fromtext()` construye un contenedor con filas para cada línea del archivo.

In [9]:
personas = etl.fromtext('./dataset/personas.txt')
print(personas)

+--------------------+
| lines              |
+====================+
| María Perez,18,F   |
+--------------------+
| Juan Pino,25,M     |
+--------------------+
| Roberto Pinto,17,M |
+--------------------+



Para tratar con la estructura del archivo, puede posprocesarse la tabla por medio de __expresiones regulares__.

In [10]:
tabla_personas = etl.capture(personas, 'lines', '(.*),(.*),(.*)', ['nombre', 'edad', 'genero'])
tabla_personas

nombre,edad,genero
María Perez,18,F
Juan Pino,25,M
Roberto Pinto,17,M


### Archivos CSV
De acuerdo a su especificación [RFC 4180](https://tools.ietf.org/html/rfc4180), el formato:

- Es un __archivo de texto en formato plano__.
- Almacena datos como en __registros__, cada uno de los cuales está localizado en una línea separada por un salto de línea.
- Cada registro almacena datos tabulares __con valores separados por coma__, aunque también pueden estar separados por otro símbolo, como el punto y coma `;`, espacios, tabulaciones (`\t`), etc.
- Comunmente se utiliza para el __intercambio de datos__.

![Dato](./img/almacenamiento_dato.svg)

<dl class="row dl-horizontal">
    <dt class="text-right">Registro:</dt>
    <dd>colección de campos logicamente relacionados que pueden ser tratados en un programa como una unidad.</dd>
    <dt class="text-right">Campo:</dt>
    <dd>item o elemento de datos, considerado la unidad mínima de información de un registro. Generalmente se caracterizan por su tipo (e.g. entero, lógico).</dd>
</dl>

__Para extraer datos desde archivos CSV__ se permite incluir los siguientes parámetros como _keywords_:
- `header` encabezados de columnas como una lista.
- `delimiter` símbolo delimitador de campos.

In [12]:
niveles = etl.fromcsv('./dataset/niveles.csv', delimiter=',')
niveles

Fecha,Nivel (m)
2012-01-23,64.59
2012-05-23,64.79
2012-07-30,64.35
2012-09-10,64.77
2012-11-15,68.05


### Archivos Hyper Text Markup Language (HTML)

[HTML](https://www.w3schools.com/html/html_intro.asp) es un medio para describir la __estructura y el código__ para la definición del contenido de una página web (e.g. texto, imágenes, videos).

El [_Document Object Model_ (DOM)](https://www.w3schools.com/whatis/whatis_htmldom.asp) define la estructura básica de un documento HTML a partir de la cual se puede construir un documento, navegar sobre este, añadir, modificar o eliminar elementos.

```html
<html>
    <head>
        <title>Título de la página</title>
    </head>
    <body>
        <h1>Encabezado</h1>
        <p>Este es un párrafo.</p>
        <p>Este es otro párrafo.</p>
        <table>
            <tr>
                <td>Celda</td>
            </tr>
        </table>
    </body>
</html>
```

Un _browser_ (e.g. IE, Firefox, Google Chrome) lee e interpreta los componentes y los interpreta para representarlos al usuario en un formato legible.

La __librería [Pandas](https://pandas.pydata.org)__ es una biblioteca de código de abierto que proporciona estructuras de datos de alto rendimiento y herramientas para el análisis de datos.

La función `.read_html(url)` __lee tablas en lenguaje HTML__ y retorna un objeto `dataframe`. En el siguiente ejemplo se muestra la extracción de la tabla de sismos de Centro Sismológico Nacional de la Universidad de Chile en la Red Sismológica Mundial publicada en su [portal web](http://www.sismologia.cl/links/ultimos_sismos.html).

In [13]:
import pandas as pd

url = 'http://www.sismologia.cl/ultimos_sismos.html'
tablas = pd.read_html(url)
sismos_df = tablas[0]

sismos_df

,Fecha Local,Fecha UTC,Latitud,Longitud,Profundidad [Km],Magnitud,Referencia Geográfica
0,2021-10-20 17:56:08,2021-10-20 20:56:08,-34.421,-73.750,25.3,2.7 Ml,159 km al NO de Constitución
1,2021-10-20 17:20:20,2021-10-20 20:20:20,-21.989,-69.797,81.4,3.1 Ml,42 km al N de María Elena
2,2021-10-20 17:16:46,2021-10-20 20:16:46,-29.776,-72.044,37.0,3.3 Ml,75 km al NO de Tongoy
3,2021-10-20 17:07:09,2021-10-20 20:07:09,-20.594,-69.192,111.7,2.7 Ml,18 km al SE de Pica
4,2021-10-20 16:53:17,2021-10-20 19:53:17,-29.778,-72.006,31.4,2.6 Ml,72 km al NO de Tongoy
5,2021-10-20 14:59:50,2021-10-20 17:59:50,-22.112,-68.721,116.5,3.1 Ml,45 km al NE de Calama
6,2021-10-20 11:36:34,2021-10-20 14:36:34,-31.111,-72.472,33.0,2.9 Ml,102 km al O de Canela Baja
7,2021-10-20 11:12:48,2021-10-20 14:12:48,-24.111,-67.564,198.3,3.2 Ml,67 km al SE de Socaire
8,2021-10-20 10:15:03,2021-10-20 13:15:03,-29.629,-71.352,58.1,3.2 Ml,20 km al SO de La Higuera
9,2021-10-20 08:37:21,2021-10-20 11:37:21,-22.419,-67.818,191.8,2.9 Ml,68 km al NE de San Pedro de Atacama


Por medio de la función `.fromdataframe()` el objeto __`dataframe` es convertido en un cotenedor__ (tabla). 

In [15]:
sismos_tabla = etl.fromdataframe(sismos_df)
print(sismos_tabla)

+---------------------+---------------------+---------+----------+------------------+----------+------------------------------+
| Fecha Local         | Fecha UTC           | Latitud | Longitud | Profundidad [Km] | Magnitud | Referencia Geográfica        |
+=====================+=====================+=========+==========+==================+==========+==============================+
| 2021-10-20 17:56:08 | 2021-10-20 20:56:08 | -34.421 |   -73.75 |             25.3 | 2.7 Ml   | 159 km al NO de Constitución |
+---------------------+---------------------+---------+----------+------------------+----------+------------------------------+
| 2021-10-20 17:20:20 | 2021-10-20 20:20:20 | -21.989 |  -69.797 |             81.4 | 3.1 Ml   | 42 km al N de María Elena    |
+---------------------+---------------------+---------+----------+------------------+----------+------------------------------+
| 2021-10-20 17:16:46 | 2021-10-20 20:16:46 | -29.776 |  -72.044 |             37.0 | 3.3 Ml   | 75 km a

In [16]:
print(etl.cut(sismos_tabla, 'Fecha Local', 'Magnitud', 'Referencia Geográfica'))

+---------------------+----------+------------------------------+
| Fecha Local         | Magnitud | Referencia Geográfica        |
+=====================+==========+==============================+
| 2021-10-20 17:56:08 | 2.7 Ml   | 159 km al NO de Constitución |
+---------------------+----------+------------------------------+
| 2021-10-20 17:20:20 | 3.1 Ml   | 42 km al N de María Elena    |
+---------------------+----------+------------------------------+
| 2021-10-20 17:16:46 | 3.3 Ml   | 75 km al NO de Tongoy        |
+---------------------+----------+------------------------------+
| 2021-10-20 17:07:09 | 2.7 Ml   | 18 km al SE de Pica          |
+---------------------+----------+------------------------------+
| 2021-10-20 16:53:17 | 2.6 Ml   | 72 km al NO de Tongoy        |
+---------------------+----------+------------------------------+
...



### Archivos eXtensible Markup Language (XML)

[XML](https://www.w3schools.com/xml/default.asp) es un lenguaje de marcado independiente, diseñado para el almacenamiento y transporte de datos.

- Almacena datos en __texto en formato plano__.
- Es __autodescriptivo__.
- Sólo es información etiquetada.
- Se debe construir la lógica para __enviar, recibir, almacenar y representar__ el contenido.
- Es extensible.

El siguiente es un ejemplo del contenido de un archivo en formato XML.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<CATALOG>
	<CD>
		<TITLE>Empire Burlesque</TITLE>
		<ARTIST>Bob Dylan</ARTIST>
		<COUNTRY>USA</COUNTRY>
		<COMPANY>Columbia</COMPANY>
		<PRICE>10.90</PRICE>
		<YEAR>1985</YEAR>
	</CD>
	<CD>
		<TITLE>Hide your heart</TITLE>
		<ARTIST>Bonnie Tyler</ARTIST>
		<COUNTRY>UK</COUNTRY>
		<COMPANY>CBS Records</COMPANY>
		<PRICE>9.90</PRICE>
		<YEAR>1988</YEAR>
	</CD>
    :
    .
</CATALOG>
```

__La extracción de datos desde un archivo XML__ pueden realizarse proporcionando una asignación de nombres de campo a rutas de los elementos.

In [19]:
catalogo = etl.fromxml('./dataset/cd_catalog.xml', 'CD', 
                       {'cia': 'COMPANY', 'artista': 'ARTIST', 'pais': 'COUNTRY', 'titulo': 'TITLE'})
print(catalogo)

+-----------------+----------------+------+---------------------+
| artista         | cia            | pais | titulo              |
+=================+================+======+=====================+
| Bob Dylan       | Columbia       | USA  | Empire Burlesque    |
+-----------------+----------------+------+---------------------+
| Bonnie Tyler    | CBS Records    | UK   | Hide your heart     |
+-----------------+----------------+------+---------------------+
| Dolly Parton    | RCA            | USA  | Greatest Hits       |
+-----------------+----------------+------+---------------------+
| Gary Moore      | Virgin records | UK   | Still got the blues |
+-----------------+----------------+------+---------------------+
| Eros Ramazzotti | BMG            | EU   | Eros                |
+-----------------+----------------+------+---------------------+
...



### Archivos JavaScript Object Notation (JSON)

[JSON](https://www.json.org) es un format ligero para el intercambio de datos.

- Almacena __datos en formato de texto__.
- Es __independiente__, pero utiliza convenciones familiares para los desarrolladores de distintos lenguajes de programación.
- Es __entendible por las personas__ y __facil de interpretar por las máquinas__.

La estructura se basa en:

- Una __colección de pares__ `name:value`. En lenguajes es realizado en un `object`, `record`, `struct`, `dictionary`, `associative array`, etc.
- Una __lista ordenada de valores__. En lenguajes de programación en realizado una matriz, vector, lista o secuencia.
- __Tipos de datos__: cadenas de texto, números, arreglos, lógicos, nulos.

```json
[
    {
    "nombre": "Juan",
    "segundo_nombre": null,
    "edad": 30,
    "ciudad": "Santiago",
    "casado": true
    },
    {
    "nombre": "Sandra",
    "segundo_nombre": "Ariel",
    "edad": 24,
    "ciudad": "Concepción",
    "casado": false
    }
]
```

La función `.fromjson()` __extrae datos desde un archivo JSON__. 
- El archivo debe contener un arreglo como objeto de nivel superior.
- Cada miembro del arreglo serña tratado como una fila de datos en el contenedor.

In [21]:
empleados = etl.fromjson('./dataset/empleados.json')
empleados

nombre,segundo_nombre,edad,ciudad,casado,telefono
Juan,None,30,Santiago,True,987654321
Sandra,Ariel,24,Concepción,False,123456789


### Archivos Microsoft Excel (XLS, XLSX)

XLS es un formato de archivo binario llamado Excel Binary File Format usado por Microsoft Excel hasta la versión 2007. Almacena, 

- Archivos de hojas de cálculo.
- Tablas de filas y columnas.

XLSX es el formato Microsoft Excel Open XML para el almacenamiento de hojas de cálculo usado por por Microsoft Excel desde la versión 2007.

__La extracción de datos desde archivos XLS o XLSX__, permite incluir los siguientes parámetros como _keywords_:

- `sheet` hoja del libro.
- `row_offset` fila inicial de la extracción.
- `column_offset`columna inicial de la extracción.

In [22]:
niveles = etl.fromxlsx('./dataset/niveles.xlsx')
niveles

Fecha,Nivel (m)
2012-01-23,64.59
2012-05-23,64.79
2012-07-30,64.35
2012-09-10,64.77
2012-11-15,68.05
